# User-User content based filtering experiments
### 7/22/23
Source: https://medium.com/grabngoinfo/recommendation-system-user-based-collaborative-filtering-a2e76e3e15c4

### Imports

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity

import sys

### Data

In [2]:
student_data = pd.read_excel("dataset/StudentInformationTable.xlsx")
course_data = pd.read_excel("dataset/CourseInformationTable.xlsx")
career_data = pd.read_excel("dataset/CourseSelectionTable.xlsx")

In [3]:
display(student_data.describe())
display(student_data.head(5))

,StudentId,EnrollmentYear
count,4568.000000,4568.000000
mean,2284.500000,2018.295972
std,1318.812344,1.191886
min,1.000000,2000.000000
25%,1142.750000,2018.000000
50%,2284.500000,2019.000000
75%,3426.250000,2019.000000
max,4568.000000,2020.000000


,StudentId,EnrollmentYear,Education,Major
0,1115,2018,Undergraduate,Biological Science
1,1108,2018,Undergraduate,Biological Science
2,1192,2018,Undergraduate,Urban and Rural Planning
3,1193,2018,Undergraduate,Urban and Rural Planning
4,1293,2018,Undergraduate,World History


In [4]:
display(course_data.describe())
display(course_data.head(5))

,CourseId,Grade
count,5591.000000,5225.000000
mean,2796.000000,2.436842
std,1614.127009,0.939362
min,1.000000,0.000000
25%,1398.500000,2.000000
50%,2796.000000,2.000000
75%,4193.500000,3.000000
max,5591.000000,12.000000


,CourseId,CourseName,College,Type,Grade,Prerequisite,Introduction
0,362,Fascinating Robot,College of Engineering,Whole school optional,2.0,NaN,This course is open to all students in the sch...
1,1045,Introduction to Seismology,School of Earth and Space Sciences,General elective course,2.0,NaN,This course is a quality education general cou...
2,1647,Speeches and oral cultures in China,Department of Chinese Language and Literature,Whole school optional,2.0,NaN,The course is based on the introduction and re...
3,1830,Modern Chinese History,Department of History,Required major,4.0,ancient Chinese history,This course is based on a large number of orig...
4,1834,Chinese Historiography,Department of History,optional,3.0,NaN,This course is a compulsory course for undergr...


In [5]:
display(career_data.describe())
display(career_data.head(5))

,StudentId,Semester,CourseId,Score
count,208949.000000,208941.000000,208949.000000,149223.000000
mean,1878.772590,1.505344,2578.111147,81.154792
std,1245.936537,0.532999,1732.925391,13.841620
min,1.000000,1.000000,1.000000,0.000000
25%,778.000000,1.000000,750.000000,78.000000
50%,1695.000000,1.000000,2569.000000,84.000000
75%,2914.000000,2.000000,4151.000000,90.000000
max,4568.000000,3.000000,5591.000000,100.000000


,StudentId,AcademicYear,Semester,CourseId,CourseName,CourseCollege,Score
0,1115,18-19,1.0,146,Advanced Mathematics (B) (1),National School of Development,81.0
1,1115,18-19,1.0,148,Problem-solving on Higher Mathematics (B),School of Economics,NaN
2,1115,18-19,1.0,654,General Chemistry Practice,College of Engineering,NaN
3,1115,18-19,1.0,681,General Chemistry (B),Department of Medicine Teaching office,72.0
4,1115,18-19,1.0,684,General Chemistry Lab.（B）,Department of Medicine Teaching office,83.5


Note: We think the missing grades are pass/fail classes but this is speculation

### Data cleaning/merging

In [6]:
career_data_clean = career_data.dropna()

In [7]:
print("Number of students in data:  ", career_data_clean.StudentId.nunique())
print("Range of scores:             ", career_data_clean.Score.min(), career_data_clean.Score.max())
print("Unique scores in dataset:    ")
print(np.array(sorted(career_data_clean.Score.unique())))

Number of students in data:   4546
Range of scores:              0.0 100.0
Unique scores in dataset:    
[  0.    1.    1.5   2.    2.5   3.    3.5   4.    5.    5.5   6.    7.
   7.5   8.    9.    9.5  10.   11.   12.   12.5  13.   14.   14.5  15.
  16.   16.5  17.   18.   19.   19.5  20.   20.5  21.   22.   22.5  23.
  23.5  23.6  24.   25.   25.5  26.   27.   27.5  28.   28.5  29.   30.
  30.5  31.   32.   33.   34.   34.5  35.   35.5  36.   36.5  37.   37.5
  38.   38.5  39.   39.5  40.   40.5  41.   41.5  42.   42.5  43.   43.5
  44.   44.5  45.   45.5  46.   46.5  47.   47.5  48.   48.5  49.   49.5
  50.   50.5  51.   51.5  52.   52.5  53.   53.5  54.   54.5  55.   55.5
  56.   56.5  57.   57.5  58.   59.   59.5  60.   60.5  61.   61.5  62.
  62.5  63.   63.5  64.   64.5  65.   65.5  66.   66.5  66.6  67.   67.5
  68.   68.5  69.   69.5  70.   70.5  70.7  71.   71.5  72.   72.5  73.
  73.5  74.   74.5  75.   75.5  76.   76.5  77.   77.5  78.   78.5  79.
  79.5  80.   80.5  80.8  

In [8]:
career_student_data = pd.merge(career_data_clean, student_data, 'inner', 'StudentId')

In [9]:
display(career_student_data.describe())
display(career_student_data.head(5))

,StudentId,Semester,CourseId,Score,EnrollmentYear
count,149021.000000,149021.000000,149021.000000,149021.000000,149021.000000
mean,1631.926044,1.430362,2723.731306,81.136139,2017.849874
std,1163.655105,0.527811,1690.240923,13.840801,1.213044
min,1.000000,1.000000,2.000000,0.000000,2000.000000
25%,617.000000,1.000000,1103.000000,78.000000,2017.000000
50%,1411.000000,1.000000,2740.000000,84.000000,2018.000000
75%,2592.000000,2.000000,4152.000000,90.000000,2019.000000
max,4568.000000,3.000000,5591.000000,100.000000,2020.000000


,StudentId,AcademicYear,Semester,CourseId,CourseName,CourseCollege,Score,EnrollmentYear,Education,Major
0,1115,18-19,1.0,146,Advanced Mathematics (B) (1),National School of Development,81.0,2018,Undergraduate,Biological Science
1,1115,18-19,1.0,681,General Chemistry (B),Department of Medicine Teaching office,72.0,2018,Undergraduate,Biological Science
2,1115,18-19,1.0,684,General Chemistry Lab.（B）,Department of Medicine Teaching office,83.5,2018,Undergraduate,Biological Science
3,1115,18-19,1.0,748,Physiology,College of Life Sciences,85.0,2018,Undergraduate,Biological Science
4,1115,18-19,1.0,844,Physiology Lab.,College of Life Sciences,75.0,2018,Undergraduate,Biological Science


### Data partitioning
We should partition data on students, not on individual classes they have taken, so instead of partitioning by selecting random rows we will be selecting random students

In [10]:
all_student_ids = career_student_data.StudentId.unique()
training_students = np.random.choice(all_student_ids, int(all_student_ids.size * .8), False)
testing_students = np.array([i for i in all_student_ids if i not in training_students])
training_data = career_student_data[career_student_data["StudentId"].isin(training_students)]
testing_data = career_student_data[career_student_data["StudentId"].isin(testing_students)]
print("Number of total students:    ", all_student_ids.size)
print("Number of training students: ", training_students.size)
print("Number of testing students:  ", testing_students.size)
display(training_data.describe())
display(testing_data.describe())

Number of total students:     4546
Number of training students:  3636
Number of testing students:   910


,StudentId,Semester,CourseId,Score,EnrollmentYear
count,119340.000000,119340.000000,119340.000000,119340.000000,119340.000000
mean,1622.408723,1.430501,2724.036618,81.048897,2017.845735
std,1156.506090,0.527879,1693.828128,13.994918,1.217820
min,1.000000,1.000000,2.000000,0.000000,2000.000000
25%,626.000000,1.000000,1101.000000,78.000000,2017.000000
50%,1389.000000,1.000000,2742.000000,84.000000,2018.000000
75%,2566.000000,2.000000,4152.000000,90.000000,2019.000000
max,4568.000000,3.000000,5591.000000,100.000000,2020.000000


,StudentId,Semester,CourseId,Score,EnrollmentYear
count,29681.000000,29681.000000,29681.000000,29681.000000,29681.000000
mean,1670.192851,1.429804,2722.503723,81.486918,2017.866514
std,1191.219384,0.527543,1675.767934,13.197398,1.193524
min,7.000000,1.000000,21.000000,0.000000,2014.000000
25%,585.000000,1.000000,1166.000000,78.000000,2017.000000
50%,1489.000000,1.000000,2737.000000,84.000000,2018.000000
75%,2680.000000,2.000000,4152.000000,90.000000,2019.000000
max,4557.000000,3.000000,5591.000000,100.000000,2020.000000


Our means and standard deviations are looking pretty comparable, we could do a t-test to affirm the null hypothesis that our training and testing are 99% probably not statistically different. But I will do this later because we rlly don't need it

### Student-Course Matrix

In [11]:
print("Our training matrix will be a", training_data.StudentId.nunique(), "by", training_data.CourseId.nunique(), "table")
print("Assuming we store a float at each point for the score our table will occupy:", training_data.StudentId.nunique() * training_data.CourseId.nunique() * sys.getsizeof(training_data.Score[0]) /1000000000, "GB")

Our training matrix will be a 3636 by 3537 table
Assuming we store a float at each point for the score our table will occupy: 0.411537024 GB


This is feasable to hold in memory, but definitely a huge table!

In [20]:
training_matrix = training_data.pivot_table(index="StudentId", columns="CourseId", values="Score")
print('Sample courses taken by the first student in our matrix, because the matrix is incredibly sparse\n', training_matrix.iloc[0].dropna())
display(training_matrix.head(5))

Sample courses taken by the first student in our matrix, because the matrix is incredibly sparse
 CourseId
362     72.0
556     75.0
649     61.0
1045    83.0
1102    72.0
        ... 
4355    82.0
4357    80.0
4360    78.0
4658    60.0
4662    43.0
Name: 1, Length: 70, dtype: float64


CourseId,2,6,7,9,10,11,15,20,21,27,...,5579,5580,5581,5582,5584,5587,5588,5589,5590,5591
StudentId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### GPA Time:
Because some students get a higher score than others we need to account for a students GPA so that are final scores are not biased by higher scoring students. To do this we will be normalizing our data by subtracting by the gpa and scaling variance to be on a scale between -1 and 1

In [33]:
train_mat_norm = training_matrix.subtract(training_matrix.mean(axis=1), axis='rows')
train_mat_norm = train_mat_norm.divide(train_mat_norm.max(axis=1) * 2, axis='rows')
train_mat_norm = train_mat_norm.add(.5)
print('Sample courses taken by the first student in our matrix, because the matrix is incredibly sparse\n', train_mat_norm.iloc[0].dropna())

Sample courses taken by the first student in our matrix, because the matrix is incredibly sparse
 CourseId
362     0.596128
556     0.663440
649     0.349317
1045    0.842939
1102    0.596128
          ...   
4355    0.820501
4357    0.775627
4360    0.730752
4658    0.326880
4662   -0.054555
Name: 1, Length: 70, dtype: float64


#### Similarity metrics:
We will be using cosin similarity to get a mearsure of how similar users are

In [39]:
student_similarity = pd.DataFrame(cosine_similarity(train_mat_norm.fillna(0)), index=train_mat_norm.index, columns=train_mat_norm.index)
display(student_similarity.head(5))

StudentId,1,2,3,4,5,6,8,9,10,11,...,4554,4555,4558,4559,4560,4564,4565,4566,4567,4568
StudentId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.064512,-0.006539,0.027069,0.029729,-0.015985,-0.029645,0.060707,0.047180,0.533358,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.064512,1.000000,0.113149,0.050077,0.109801,0.127561,0.133994,0.103148,0.032307,0.162479,...,0.0,0.0,0.0,0.0,0.0,0.023037,-0.057752,0.018486,-0.007983,-0.006334
3,-0.006539,0.113149,1.000000,0.183446,0.214082,0.244421,0.491898,0.033626,0.028410,0.054114,...,0.0,0.0,0.0,0.0,0.0,0.028608,0.065402,0.040510,0.042713,0.030200
4,0.027069,0.050077,0.183446,1.000000,0.182722,0.137434,0.168266,0.024609,0.000000,0.013404,...,0.0,0.0,0.0,0.0,0.0,0.019502,0.018515,0.013353,0.021760,-0.010340
5,0.029729,0.109801,0.214082,0.182722,1.000000,0.163009,0.199672,0.017835,0.057401,0.039882,...,0.0,0.0,0.0,0.0,0.0,0.020815,-0.041490,0.021110,-0.003846,0.003833


In [43]:
print('Median user similarity:', student_similarity.median().mean())

Median user similarity: 0.08242501714596241


## Recomendation System
We will now create a function that takes in some user data and will output